In [1]:
import pandas as pd
import tweepy           
from tweepy import OAuthHandler
from tweepy import Stream
from textblob import TextBlob 

### Scrapping des données de Twitter

In [2]:
#Twitter API mots de passe
consumer_key = 'ZGC3TfWp8gXwwttH0lIYCsAKI'
consumer_secret = 'YVlmZTpeEYy8mjRXpkKlXvne4HrTOEDnmN9wK56c6Xjq31loOr'
access_token = '769589675698913284-A2eHB9MPld41p2yusKAM0V3LFfzt6Gy'
access_token_secret = 'ILlFouavDwlN1SDKJ2pQcEIgOdgT6Erx0piCsHrKaTXk6'

In [3]:
OAUTH_KEYS = {'consumer_key':consumer_key, 'consumer_secret':consumer_secret,
 'access_token_key':access_token, 'access_token_secret':access_token_secret}
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
search = tweepy.Cursor(api.search, q='#hyperloop').items(500) # ici on spécifie le mot clé

In [ ]:
# Creation des listes pour chaque tweet
it= []
sn = []
text = []
timestamp =[]
for tweet in search:
    #print(tweet.id_str,tweet.user.screen_name, tweet.created_at, tweet.text)
    it.append(tweet.id_str)
    sn.append(tweet.user.screen_name)
    text.append(tweet.text)
    timestamp.append(tweet.created_at)
    blob = TextBlob(tweet.text)
    #print(blob.sentiment.polarity)


# Converstion des listes vers des dataframe (objet pandas)
df = pd.DataFrame()
df['it'] = it
df['sn'] = sn
df['text'] = text
df['timestamp'] = timestamp



# Pour décaler les temps par rapport à l'heure locale aux états unis
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['EST'] = df['timestamp'] - pd.Timedelta(hours=5)
df['EST'] = pd.to_datetime(df['EST'])
df.to_csv('hyperloop.csv',index = False,encoding='utf-8')
allNames = list(df['sn'].unique())


# Initialize dataframe of users that will hold the edge relationships
dfUsers = pd.DataFrame()
dfUsers['userFromName'] =[]
dfUsers['userFromId'] =[]
dfUsers['userToId'] = []
count = 0
len(allNames)


## Preprocessing

In [5]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
nltk.download('stopwords')
import string
import re
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jileni\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df = pd.read_csv('hyperloop.csv')

In [7]:
df 

it               sn  \
0    1062833305257689088    OntScienceCtr   
1    1062829590463299588          ddskier   
2    1062823616906178560   HelmutRedecker   
3    1062823053263101952    ShakkhaEsprit   
4    1062822450361180160    ShakkhaEsprit   
5    1062814135308730368          epfloop   
6    1062813952655179776  mohamed_abuKhal   
7    1062810344370905094  MechatronicGmbH   
8    1062804765573345280  J_Degli_Esposti   
9    1062795138500583424    RoblesNatalia   
10   1062793676571717632      FarakhJavid   
11   1062793243421630465      RMHyperloop   
12   1062787305029726210    TreeHouseReal   
13   1062779305988706304     _breannafaye   
14   1062776385356189696        CoryTyler   
15   1062775434557972480        rLoopTeam   
16   1062772327048372224       Wangwowuwo   
17   1062771867633811456           eloace   
18   1062771858104352768     MIGHyperloop   
19   1062770948166823941     6abdalaziz66   
20   1062768368242040834     MIGHyperloop   
21   1062767258718687239     MIGHyperloop   
22   1062767239437398016     EcoServiceCA   
23   1062767131115302912   TEConnectivity   
24   1062766865859178496        TiyaniTee   
25   1062764184721678336  loiseau_martine   
26   1062761516624154625         tboitier   
27   1062761394196615169       SGendron79   
28   1062761171269378049     TransPod_Inc   
29   1062752557464993800     RSComponents   
..                   ...              ...   
470  1061317164991500293         EPFL_SPS   
471  1061314075190280193       DirkSchart   
472  1061309837479424000   MarcoCHardware   
473  1061309459832684544       mfherrador   
474  1061308326510780417  LuisJorgeAlloza   
475  1061307508969652224    trespalabras5   
476  1061307338718674944         GRAUSAFL   
477  1061298377336504320       EPFLevents   
478  1061297706390470658   inthehyperloop   
479  1061293792488288257             Kyol   
480  1061289731948978176       javimateog   
481  1061289690974904323      Startup_VLC   
482  1061289115306602498   valenciactiva_   
483  1061288045247426560     Carlos_Storm   
484  1061286814890246145   Manuel_Ortizpr   
485  1061286038302310400         titemure   
486  1061285340219142145    bufetaguilera   
487  1061278736887283712          zeleros   
488  1061277617599373312    jasphervaldez   
489  1061275845594136577      Dameunabeca   
490  1061275621009965057  Prempal67579252   
491  1061274147588837376  CamaradaBakunin   
492  1061273810400432129      brucknerite   
493  1061261024429400065       2601Manish   
494  1061260875263180801         i_amzaid   
495  1061257934695321601     powerloopxyz   
496  1061257780789567494     powerkingxyz   
497  1061248622031966208       AlainMouel   
498  1061248439990841345       AlainMouel   
499  1061243953461321728     Poonam_Datta   

                                                  text            timestamp  \
0    RT @TransPod_Inc: From inception to #innovatio...  2018-11-14 22:23:01   
1    Virgin #Hyperloop One’s New CEO Could Make a W...  2018-11-14 22:08:15   
2    RT @epfloop: We have something very exciting c...  2018-11-14 21:44:31   
3    RT @keruffle: #Hyperloop #maglev above existin...  2018-11-14 21:42:16   
4    RT @funimag: Hyperloop "Une formidable escroqu...  2018-11-14 21:39:53   
5    We have something very exciting coming your wa...  2018-11-14 21:06:50   
6    RT @SaudiProjectEN: CEO of #HyperLoop: There i...  2018-11-14 21:06:07   
7    RT @FarakhJavid: Awesome day at SEMICON Europa...  2018-11-14 20:51:46   
8    RT @EnjoyDigitAll: (#pointdevue) #Hyperloop es...  2018-11-14 20:29:36   
9    @ThierryLabro @olivcim @LADN_EU Tu as bonne mé...  2018-11-14 19:51:21   
10   Awesome day at SEMICON Europa ! #semiconductor...  2018-11-14 19:45:32   
11   RT @HardtGlobal: Proud of our latest video! "W...  2018-11-14 19:43:49   
12   New full-scale #Hyperloop passenger capsule re...  2018-11-14 19:20:13   
13   RT @TEConnectivity: The @rLoopTeam at @electro...  2018-11-14 18:48:26   
14   10 Hyperloops That Will Change The World 

In [8]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

df['Tweet_punct'] = df['text'].apply(lambda x: remove_punct(x))
df.head(10)

it               sn  \
0  1062833305257689088    OntScienceCtr   
1  1062829590463299588          ddskier   
2  1062823616906178560   HelmutRedecker   
3  1062823053263101952    ShakkhaEsprit   
4  1062822450361180160    ShakkhaEsprit   
5  1062814135308730368          epfloop   
6  1062813952655179776  mohamed_abuKhal   
7  1062810344370905094  MechatronicGmbH   
8  1062804765573345280  J_Degli_Esposti   
9  1062795138500583424    RoblesNatalia   

                                                text            timestamp  \
0  RT @TransPod_Inc: From inception to #innovatio...  2018-11-14 22:23:01   
1  Virgin #Hyperloop One’s New CEO Could Make a W...  2018-11-14 22:08:15   
2  RT @epfloop: We have something very exciting c...  2018-11-14 21:44:31   
3  RT @keruffle: #Hyperloop #maglev above existin...  2018-11-14 21:42:16   
4  RT @funimag: Hyperloop "Une formidable escroqu...  2018-11-14 21:39:53   
5  We have something very exciting coming your wa...  2018-11-14 21:06:50   
6  RT @SaudiProjectEN: CEO of #HyperLoop: There i...  2018-11-14 21:06:07   
7  RT @FarakhJavid: Awesome day at SEMICON Europa...  2018-11-14 20:51:46   
8  RT @EnjoyDigitAll: (#pointdevue) #Hyperloop es...  2018-11-14 20:29:36   
9  @ThierryLabro @olivcim @LADN_EU Tu as bonne mé...  2018-11-14 19:51:21   

                   EST                                        Tweet_punct  
0  2018-11-14 17:23:01  RT TransPodInc From inception to innovation ou...  
1  2018-11-14 17:08:15  Virgin Hyperloop One’s New CEO Could Make a Wo...  
2  2018-11-14 16:44:31  RT epfloop We have something very exciting com...  
3  2018-11-14 16:42:16  RT keruffle Hyperloop maglev above existing hi...  
4  2018-11-14 16:39:53  RT funimag Hyperloop Une formidable escroqueri...  
5  2018-11-14 16:06:50  We have something very exciting coming your wa...  
6  2018-11-14 16:06:07  RT SaudiProjectEN CEO of HyperLoop There is a ...  
7  2018-11-14 15:51:46  RT FarakhJavid Awesome day at SEMICON Europa  ...  
8  2018-11-14 15:29:36  RT EnjoyDigitAll pointdevue Hyperloop est une ...  
9  2018-11-14 14:51:21  ThierryLabro olivcim LADNEU Tu as bonne mémoir...

In [9]:
df= df[['sn','text','Tweet_punct']]

In [10]:
df.head(10)

sn                                               text  \
0    OntScienceCtr  RT @TransPod_Inc: From inception to #innovatio...   
1          ddskier  Virgin #Hyperloop One’s New CEO Could Make a W...   
2   HelmutRedecker  RT @epfloop: We have something very exciting c...   
3    ShakkhaEsprit  RT @keruffle: #Hyperloop #maglev above existin...   
4    ShakkhaEsprit  RT @funimag: Hyperloop "Une formidable escroqu...   
5          epfloop  We have something very exciting coming your wa...   
6  mohamed_abuKhal  RT @SaudiProjectEN: CEO of #HyperLoop: There i...   
7  MechatronicGmbH  RT @FarakhJavid: Awesome day at SEMICON Europa...   
8  J_Degli_Esposti  RT @EnjoyDigitAll: (#pointdevue) #Hyperloop es...   
9    RoblesNatalia  @ThierryLabro @olivcim @LADN_EU Tu as bonne mé...   

                                         Tweet_punct  
0  RT TransPodInc From inception to innovation ou...  
1  Virgin Hyperloop One’s New CEO Could Make a Wo...  
2  RT epfloop We have something very exciting com...  
3  RT keruffle Hyperloop maglev above existing hi...  
4  RT funimag Hyperloop Une formidable escroqueri...  
5  We have something very exciting coming your wa...  
6  RT SaudiProjectEN CEO of HyperLoop There is a ...  
7  RT FarakhJavid Awesome day at SEMICON Europa  ...  
8  RT EnjoyDigitAll pointdevue Hyperloop est une ...  
9  ThierryLabro olivcim LADNEU Tu as bonne mémoir...

In [12]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

df['Tweet_tokenized'] = df['Tweet_punct'].apply(lambda x: tokenization(x.lower()))
df.head()

c:\users\jileni\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


sn                                               text  \
0   OntScienceCtr  RT @TransPod_Inc: From inception to #innovatio...   
1         ddskier  Virgin #Hyperloop One’s New CEO Could Make a W...   
2  HelmutRedecker  RT @epfloop: We have something very exciting c...   
3   ShakkhaEsprit  RT @keruffle: #Hyperloop #maglev above existin...   
4   ShakkhaEsprit  RT @funimag: Hyperloop "Une formidable escroqu...   

                                         Tweet_punct  \
0  RT TransPodInc From inception to innovation ou...   
1  Virgin Hyperloop One’s New CEO Could Make a Wo...   
2  RT epfloop We have something very exciting com...   
3  RT keruffle Hyperloop maglev above existing hi...   
4  RT funimag Hyperloop Une formidable escroqueri...   

                                     Tweet_tokenized  
0  [rt, transpodinc, from, inception, to, innovat...  
1  [virgin, hyperloop, one, s, new, ceo, could, m...  
2  [rt, epfloop, we, have, something, very, excit...  
3  [rt, keruffle, hyperloop, maglev, above, exist...  
4  [rt, funimag, hyperloop, une, formidable, escr...

In [13]:
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
df['Tweet_nonstop'] = df['Tweet_tokenized'].apply(lambda x: remove_stopwords(x))
df.head(10)

c:\users\jileni\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


sn                                               text  \
0    OntScienceCtr  RT @TransPod_Inc: From inception to #innovatio...   
1          ddskier  Virgin #Hyperloop One’s New CEO Could Make a W...   
2   HelmutRedecker  RT @epfloop: We have something very exciting c...   
3    ShakkhaEsprit  RT @keruffle: #Hyperloop #maglev above existin...   
4    ShakkhaEsprit  RT @funimag: Hyperloop "Une formidable escroqu...   
5          epfloop  We have something very exciting coming your wa...   
6  mohamed_abuKhal  RT @SaudiProjectEN: CEO of #HyperLoop: There i...   
7  MechatronicGmbH  RT @FarakhJavid: Awesome day at SEMICON Europa...   
8  J_Degli_Esposti  RT @EnjoyDigitAll: (#pointdevue) #Hyperloop es...   
9    RoblesNatalia  @ThierryLabro @olivcim @LADN_EU Tu as bonne mé...   

                                         Tweet_punct  \
0  RT TransPodInc From inception to innovation ou...   
1  Virgin Hyperloop One’s New CEO Could Make a Wo...   
2  RT epfloop We have something very exciting com...   
3  RT keruffle Hyperloop maglev above existing hi...   
4  RT funimag Hyperloop Une formidable escroqueri...   
5  We have something very exciting coming your wa...   
6  RT SaudiProjectEN CEO of HyperLoop There is a ...   
7  RT FarakhJavid Awesome day at SEMICON Europa  ...   
8  RT EnjoyDigitAll pointdevue Hyperloop est une ...   
9  ThierryLabro olivcim LADNEU Tu as bonne mémoir...   

                                     Tweet_tokenized  \
0  [rt, transpodinc, from, inception, to, innovat...   
1  [virgin, hyperloop, one, s, new, ceo, could, m...   
2  [rt, epfloop, we, have, something, very, excit...   
3  [rt, keruffle, hyperloop, maglev, above, exist...   
4  [rt, funimag, hyperloop, une, formidable, escr...   
5  [we, have, something, very, exciting, coming, ...   
6  [rt, saudiprojecten, ceo, of, hyperloop, there...   
7  [rt, farakhjavid, awesome, day, at, semicon, e...   
8  [rt, enjoydigitall, pointdevue, hyperloop, est...   
9  [thierrylabro, olivcim, ladneu, tu, as, bonne,...   

                                       Tweet_nonstop  
0  [rt, transpodinc, inception, innovation, cto, ...  
1  [virgin, hyperloop, one, new, ceo, could, make...  
2  [rt, epfloop, something, exciting, coming, way...  
3  [rt, keruffle, hyperloop, maglev, existing, hi...  
4  [rt, funimag, hyperloop, une, formidable, escr...  
5  [something, exciting, coming, way, soon, canno...  
6  [rt, saudiprojecten, ceo, hyperloop, common, i...  
7  [rt, farakhjavid, awesome, day, semicon, europ...  
8  [rt, enjoydigitall, pointdevue, hyperloop, est...  
9  [thierrylabro, olivcim, ladneu, tu, bonne, mém...

In [14]:
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

df['Tweet_stemmed'] = df['Tweet_nonstop'].apply(lambda x: stemming(x))
df.head()

sn                                               text  \
0   OntScienceCtr  RT @TransPod_Inc: From inception to #innovatio...   
1         ddskier  Virgin #Hyperloop One’s New CEO Could Make a W...   
2  HelmutRedecker  RT @epfloop: We have something very exciting c...   
3   ShakkhaEsprit  RT @keruffle: #Hyperloop #maglev above existin...   
4   ShakkhaEsprit  RT @funimag: Hyperloop "Une formidable escroqu...   

                                         Tweet_punct  \
0  RT TransPodInc From inception to innovation ou...   
1  Virgin Hyperloop One’s New CEO Could Make a Wo...   
2  RT epfloop We have something very exciting com...   
3  RT keruffle Hyperloop maglev above existing hi...   
4  RT funimag Hyperloop Une formidable escroqueri...   

                                     Tweet_tokenized  \
0  [rt, transpodinc, from, inception, to, innovat...   
1  [virgin, hyperloop, one, s, new, ceo, could, m...   
2  [rt, epfloop, we, have, something, very, excit...   
3  [rt, keruffle, hyperloop, maglev, above, exist...   
4  [rt, funimag, hyperloop, une, formidable, escr...   

                                       Tweet_nonstop  \
0  [rt, transpodinc, inception, innovation, cto, ...   
1  [virgin, hyperloop, one, new, ceo, could, make...   
2  [rt, epfloop, something, exciting, coming, way...   
3  [rt, keruffle, hyperloop, maglev, existing, hi...   
4  [rt, funimag, hyperloop, une, formidable, escr...   

                                       Tweet_stemmed  
0  [rt, transpodinc, incept, innov, cto, ryan, ja...  
1  [virgin, hyperloop, one, new, ceo, could, make...  
2  [rt, epfloop, someth, excit, come, way, soon, ...  
3  [rt, keruffl, hyperloop, maglev, exist, highwa...  
4  [rt, funimag, hyperloop, une, formid, escroque...

In [15]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

df['Tweet_lemmatized'] = df['Tweet_nonstop'].apply(lambda x: lemmatizer(x))
df.head()

sn                                               text  \
0   OntScienceCtr  RT @TransPod_Inc: From inception to #innovatio...   
1         ddskier  Virgin #Hyperloop One’s New CEO Could Make a W...   
2  HelmutRedecker  RT @epfloop: We have something very exciting c...   
3   ShakkhaEsprit  RT @keruffle: #Hyperloop #maglev above existin...   
4   ShakkhaEsprit  RT @funimag: Hyperloop "Une formidable escroqu...   

                                         Tweet_punct  \
0  RT TransPodInc From inception to innovation ou...   
1  Virgin Hyperloop One’s New CEO Could Make a Wo...   
2  RT epfloop We have something very exciting com...   
3  RT keruffle Hyperloop maglev above existing hi...   
4  RT funimag Hyperloop Une formidable escroqueri...   

                                     Tweet_tokenized  \
0  [rt, transpodinc, from, inception, to, innovat...   
1  [virgin, hyperloop, one, s, new, ceo, could, m...   
2  [rt, epfloop, we, have, something, very, excit...   
3  [rt, keruffle, hyperloop, maglev, above, exist...   
4  [rt, funimag, hyperloop, une, formidable, escr...   

                                       Tweet_nonstop  \
0  [rt, transpodinc, inception, innovation, cto, ...   
1  [virgin, hyperloop, one, new, ceo, could, make...   
2  [rt, epfloop, something, exciting, coming, way...   
3  [rt, keruffle, hyperloop, maglev, existing, hi...   
4  [rt, funimag, hyperloop, une, formidable, escr...   

                                       Tweet_stemmed  \
0  [rt, transpodinc, incept, innov, cto, ryan, ja...   
1  [virgin, hyperloop, one, new, ceo, could, make...   
2  [rt, epfloop, someth, excit, come, way, soon, ...   
3  [rt, keruffl, hyperloop, maglev, exist, highwa...   
4  [rt, funimag, hyperloop, une, formid, escroque...   

                                    Tweet_lemmatized  
0  [rt, transpodinc, inception, innovation, cto, ...  
1  [virgin, hyperloop, one, new, ceo, could, make...  
2  [rt, epfloop, something, exciting, coming, way...  
3  [rt, keruffle, hyperloop, maglev, existing, hi...  
4  [rt, funimag, hyperloop, une, formidable, escr...

In [16]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [ps.stem(word) for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text

In [46]:
countVectorizer = CountVectorizer(analyzer=clean_text) 
countVector = countVectorizer.fit_transform(df['text'])
print('{} Number of tweets has {} words'.format(countVector.shape[0], countVector.shape[1]))

500 Number of tweets has 1403 words


## Topic Modelling

In [17]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jileni\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [33]:
from nltk import PorterStemmer
stemmer = PorterStemmer()

In [23]:
data_text = df[['Tweet_punct']]
data_text['index'] = data_text.index
documents = data_text

c:\users\jileni\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
documents

Tweet_punct  index
0    RT TransPodInc From inception to innovation ou...      0
1    Virgin Hyperloop One’s New CEO Could Make a Wo...      1
2    RT epfloop We have something very exciting com...      2
3    RT keruffle Hyperloop maglev above existing hi...      3
4    RT funimag Hyperloop Une formidable escroqueri...      4
5    We have something very exciting coming your wa...      5
6    RT SaudiProjectEN CEO of HyperLoop There is a ...      6
7    RT FarakhJavid Awesome day at SEMICON Europa  ...      7
8    RT EnjoyDigitAll pointdevue Hyperloop est une ...      8
9    ThierryLabro olivcim LADNEU Tu as bonne mémoir...      9
10   Awesome day at SEMICON Europa  semiconductor m...     10
11   RT HardtGlobal Proud of our latest video What ...     11
12   New fullscale Hyperloop passenger capsule reve...     12
13   RT TEConnectivity The rLoopTeam at electronica...     13
14    Hyperloops That Will Change The World  The BM...     14
15   RT TEConnectivity The rLoopTeam at electronica...     15
16   RT TEConnectivity The rLoopTeam at electronica...     16
17   This Is Awesome 👏😎 Congratulations rLoopTeam T...     17
18   RT michaelmacolino Great video on the cities w...     18
19   RT SaudiProjectEN CEO of HyperLoop There is a ...     19
20   RT DesignSparkRS Members of HYPEDHyperloop are...     20
21   RT TransPodInc From inception to innovation ou...     21
22   elonmusk of boringcompany has tweeted a video ...     22
23   The rLoopTeam at electronicaFair showcasing th...     23
24   RT RSComponents There are members from  teams ...     24
25   RT ouiautraindnuit LHyperloop une formidable e...     25
26   RT TransPodInc From inception to innovation ou...     26
27   RT TransPodInc From inception to innovation ou...     27
28   From inception to innovation our CTO Ryan Janz...     28
29   There are members from  teams who competed at ...     29
..                                                 ...    ...
470  RT titemure scientastic Hyperloop epfloop yeah...    470
471  All the best for Jay Walder Not an easy job af...    471
472  Più che SiTav io chiederei che lItalia diventi...    472
473  RT brucknerite leyendo Las fichas del dominó e...    473
474  RT brucknerite leyendo Las fichas del dominó e...    474
475  RT brucknerite leyendo Las fichas del dominó e...    475
476  RT FisherM Elon Musk’s boringcompany will focu...    476
477  RT titemure scientastic Hyperloop epfloop yeah...    477
478  RT zeleros Today Zeleros participated at Valen...    478
479  RT brucknerite leyendo Las fichas del dominó e...    479
480  RT zeleros Today Zeleros participated at Valen...    480
481  RT zeleros Today Zeleros participated at Valen...    481
482  RT zeleros Today Zeleros participated at Valen...    482
483  RT brucknerite leyendo Las fichas del dominó e...    483
484  RT ciudadintelicl ¿Has escuchado hablar sobre ...    484
485  scientastic Hyperloop epfloop yeah\r\nEPFLSPS ...    485
486  elonmusk Mr elonmusk Cannot Tesla create a bom...    486
487  Today Zeleros participated at Valencias main s...    487
488  RT TEConnectivity Join us at electronicaFair i...    488
489  RT brucknerite leyendo Las fichas del dominó e...    489
490  RT TEConnectivity Join us at electronicaFair i...    490
491  RT brucknerite leyendo Las fichas del dominó e...    491
492  leyendo Las fichas del dominó empiezan a caer ...    492
493  RT JeroenBartelse DFWs bullet train hyperloop ...    493
494  RT TEConnectivity Join us at electronicaFair i...    494
495  我有超能利省油除碳又有力\r\n powerloop 超能利•勁油力\r\n\r\nhttp...    495
496  我有超能利省油除碳又有力\r\n powerloop 超能利•勁油力\r\n\r\nhttp...    496
497            Hyperloop HauteVienne httpstcomevxSMcuE    497
498              Hyperloop HauteVienne httpstcoefHAEWE    498
499  DP World Group Chairman amp CEO Sultan Ahmed B...    499

[500 rows x 2 columns]

In [34]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [27]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))


IndexError: index 1 is out of bounds for axis 0 with size 0

In [35]:
processed_docs = documents['Tweet_punct'].map(preprocess)

In [36]:
processed_docs[:10]

0    [transpodinc, incept, innov, ryan, janzen, sho...
1       [virgin, hyperloop, wooshi, futur, real, wire]
2    [epfloop, excit, come, soon, wait, shoot, epfl...
3    [keruffl, hyperloop, maglev, exist, highway, s...
4    [funimag, hyperloop, formid, escroqueri, https...
5    [excit, come, soon, wait, shoot, epfl, hyperloop]
6    [saudiprojecten, hyperloop, common, saudiarabi...
7    [farakhjavid, awesom, semicon, europa, semicon...
8    [enjoydigital, pointdevu, hyperloop, formid, e...
9    [thierrylabro, olivcim, ladneu, bonn, mémoir, ...
Name: Tweet_punct, dtype: object

## BOW

In [37]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [39]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

215 doubt
815 httpstcoqpwhzm
39 httpstcozimtiph
1001 reim
225 nostrosamu
904 plan
125 trainsdenuit
162 lor
264 khashoggi
972 emirati
804 throw


In [40]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)


In [62]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[476]

[(0, 1), (4, 1), (5, 1), (14, 1), (45, 1)]

In [63]:
bow_doc_476 = bow_corpus[476]

for i in range(len(bow_doc_476)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_476[i][0], 
                                                     dictionary[bow_doc_476[i][0]], 
                                                     bow_doc_476[i][1]))

Word 0 ("innov") appears 1 time.
Word 4 ("elon") appears 1 time.
Word 5 ("musk") appears 1 time.
Word 14 ("video") appears 1 time.
Word 45 ("tunnel") appears 1 time.


## TF-IDF

In [64]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)


In [65]:
corpus_tfidf = tfidf[bow_corpus]


In [67]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.3884610532726922), (1, 0.92146514317703332)]


## Running LDA using Bag of Words


In [68]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [69]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.079*"caer" + 0.079*"incluido" + 0.079*"ficha" + 0.079*"leyendo" + 0.079*"cierr" + 0.079*"directivo" + 0.079*"despido" + 0.079*"empiezan" + 0.079*"factoría" + 0.079*"dominó"
Topic: 1 
Words: 0.051*"dominó" + 0.051*"brucknerit" + 0.051*"factoría" + 0.051*"empiezan" + 0.051*"despido" + 0.051*"metalwork" + 0.051*"directivo" + 0.051*"cierr" + 0.051*"leyendo" + 0.051*"ficha"
Topic: 2 
Words: 0.094*"dan" + 0.088*"formid" + 0.088*"escroqueri" + 0.084*"lhyperloop" + 0.074*"trainsdenuit" + 0.074*"plutôt" + 0.074*"investisson" + 0.071*"ouiautraindnuit" + 0.053*"musk" + 0.048*"elon"
Topic: 3 
Words: 0.422*"elonmusk" + 0.109*"escroqueri" + 0.104*"formid" + 0.029*"team" + 0.022*"dan" + 0.022*"lhyperloop" + 0.022*"ouiautraindnuit" + 0.022*"trainsdenuit" + 0.022*"investisson" + 0.022*"plutôt"
Topic: 4 
Words: 0.276*"team" + 0.180*"abudhabi" + 0.078*"join" + 0.034*"stop" + 0.034*"novemb" + 0.034*"hall" + 0.034*"booth" + 0.034*"germani" + 0.034*"mini" + 0.033*"electronicafair"
Topic: 

## LDA using TF-IDF

In [70]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [71]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.079*"electronicafair" + 0.066*"teconnect" + 0.063*"munich" + 0.057*"hall" + 0.057*"novemb" + 0.057*"stop" + 0.057*"booth" + 0.057*"germani" + 0.056*"mini" + 0.053*"join"
Topic: 1 Word: 0.159*"common" + 0.159*"talk" + 0.159*"saudiarabia" + 0.157*"mont" + 0.157*"saudiprojecten" + 0.097*"elonmusk" + 0.023*"join" + 0.009*"transpodinc" + 0.005*"innov" + 0.004*"teconnect"
Topic: 2 Word: 0.237*"virgin" + 0.157*"train" + 0.048*"saudiprojecten" + 0.047*"mont" + 0.047*"talk" + 0.047*"common" + 0.047*"saudiarabia" + 0.033*"innov" + 0.025*"transport" + 0.019*"futur"
Topic: 3 Word: 0.058*"dominó" + 0.058*"directivo" + 0.058*"empiezan" + 0.058*"caer" + 0.058*"despido" + 0.058*"cierr" + 0.058*"ficha" + 0.058*"factoría" + 0.058*"incluido" + 0.058*"leyendo"
Topic: 4 Word: 0.141*"video" + 0.114*"formid" + 0.108*"tunnel" + 0.104*"escroqueri" + 0.092*"elon" + 0.091*"musk" + 0.071*"elonmusk" + 0.057*"innov" + 0.016*"team" + 0.009*"ouiautraindnuit"
Topic: 5 Word: 0.142*"abudhabi" + 0.085*"l

## Performance evaluation by classifying sample document using LDA Bag of Words model

In [72]:
processed_docs[476]

['fisherm',
 'elon',
 'musk',
 'boringcompani',
 'focu',
 'hyperloop',
 'tunnel',
 'pedestrian',
 'cyclist',
 'video',
 'innov',
 'startup']

In [74]:
for index, score in sorted(lda_model[bow_corpus[476]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8499900102615356	 
Topic: 0.094*"dan" + 0.088*"formid" + 0.088*"escroqueri" + 0.084*"lhyperloop" + 0.074*"trainsdenuit" + 0.074*"plutôt" + 0.074*"investisson" + 0.071*"ouiautraindnuit" + 0.053*"musk" + 0.048*"elon"

Score: 0.01667262427508831	 
Topic: 0.140*"futur" + 0.127*"innov" + 0.105*"electronicafair" + 0.099*"transport" + 0.070*"mini" + 0.059*"abudhabi" + 0.053*"teconnect" + 0.025*"linstal" + 0.025*"dessai" + 0.025*"pist"

Score: 0.016668573021888733	 
Topic: 0.090*"join" + 0.086*"munich" + 0.086*"teconnect" + 0.086*"electronicafair" + 0.082*"mini" + 0.082*"germani" + 0.082*"booth" + 0.082*"hall" + 0.082*"novemb" + 0.082*"stop"

Score: 0.016667546704411507	 
Topic: 0.051*"dominó" + 0.051*"brucknerit" + 0.051*"factoría" + 0.051*"empiezan" + 0.051*"despido" + 0.051*"metalwork" + 0.051*"directivo" + 0.051*"cierr" + 0.051*"leyendo" + 0.051*"ficha"

Score: 0.016667326912283897	 
Topic: 0.184*"saudiarabia" + 0.184*"talk" + 0.184*"common" + 0.182*"saudiprojecten" + 0.182*"mont

## Performance evaluation by classifying sample document using LDA TF-IDF model

In [76]:
for index, score in sorted(lda_model_tfidf[bow_corpus[476]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.8499941229820251	 
Topic: 0.141*"video" + 0.114*"formid" + 0.108*"tunnel" + 0.104*"escroqueri" + 0.092*"elon" + 0.091*"musk" + 0.071*"elonmusk" + 0.057*"innov" + 0.016*"team" + 0.009*"ouiautraindnuit"

Score: 0.016669047996401787	 
Topic: 0.269*"transpodinc" + 0.136*"train" + 0.092*"dan" + 0.067*"musk" + 0.059*"innov" + 0.056*"elon" + 0.026*"veut" + 0.018*"tunnel" + 0.017*"video" + 0.012*"team"

Score: 0.016668302938342094	 
Topic: 0.332*"transport" + 0.201*"futur" + 0.079*"innov" + 0.041*"formid" + 0.036*"escroqueri" + 0.032*"mini" + 0.030*"electronicafair" + 0.029*"teconnect" + 0.013*"limog" + 0.011*"transpodinc"

Score: 0.0166673194617033	 
Topic: 0.237*"virgin" + 0.157*"train" + 0.048*"saudiprojecten" + 0.047*"mont" + 0.047*"talk" + 0.047*"common" + 0.047*"saudiarabia" + 0.033*"innov" + 0.025*"transport" + 0.019*"futur"

Score: 0.01666712760925293	 
Topic: 0.079*"electronicafair" + 0.066*"teconnect" + 0.063*"munich" + 0.057*"hall" + 0.057*"novemb" + 0.057*"stop" + 0.057*"

## Testing model on unseen document

In [77]:
unseen_document = 'And because without music life has no meaning'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.10000000149011612	 Topic: 0.079*"caer" + 0.079*"incluido" + 0.079*"ficha" + 0.079*"leyendo" + 0.079*"cierr"
Score: 0.10000000149011612	 Topic: 0.051*"dominó" + 0.051*"brucknerit" + 0.051*"factoría" + 0.051*"empiezan" + 0.051*"despido"
Score: 0.10000000149011612	 Topic: 0.094*"dan" + 0.088*"formid" + 0.088*"escroqueri" + 0.084*"lhyperloop" + 0.074*"trainsdenuit"
Score: 0.10000000149011612	 Topic: 0.422*"elonmusk" + 0.109*"escroqueri" + 0.104*"formid" + 0.029*"team" + 0.022*"dan"
Score: 0.10000000149011612	 Topic: 0.276*"team" + 0.180*"abudhabi" + 0.078*"join" + 0.034*"stop" + 0.034*"novemb"
Score: 0.10000000149011612	 Topic: 0.184*"saudiarabia" + 0.184*"talk" + 0.184*"common" + 0.182*"saudiprojecten" + 0.182*"mont"
Score: 0.10000000149011612	 Topic: 0.090*"join" + 0.086*"munich" + 0.086*"teconnect" + 0.086*"electronicafair" + 0.082*"mini"
Score: 0.10000000149011612	 Topic: 0.310*"transport" + 0.129*"futur" + 0.063*"formid" + 0.063*"escroqueri" + 0.050*"virgin"
Score: 0.10000000